# RAG 시스템 평가

이 노트북은 RAGAS 메트릭을 사용하여 RAG 시스템의 성능을 평가합니다.

In [ ]:
import json
from datasets import Dataset
from libs.custom_ragas import (
    evaluate,
    AnswerRelevancy,
    Faithfulness,
    ContextRecall,
    ContextPrecision
)

## 1. 평가 데이터셋 준비

In [ ]:
# JSONL 파일 로드
def load_evaluation_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# 데이터셋 로드
eval_data = load_evaluation_data('evaluation_data.jsonl')
dataset = Dataset.from_list(eval_data)

## 2. 평가 메트릭 설정

In [ ]:
# 평가할 메트릭 선택
metrics = [
    AnswerRelevancy,
    Faithfulness,
    ContextRecall,
    ContextPrecision
]

# Bedrock 모델 설정
model_id = 'anthropic.claude-3-haiku-20240307'  # 사용할 모델 ID
embed_id = 'amazon.titan-embed-text-v1'  # 임베딩 모델 ID
region = 'us-west-2'  # AWS 리전

## 3. 평가 실행

In [ ]:
# 데이터셋 형식 변환
def map_dataset(example):
    return {
        "user_input": example["question"],
        "retrieved_contexts": example["retrieved_contexts"],
        "response": example["answer"],
        "reference": example["ground_truth"]
    }

formatted_dataset = dataset.map(map_dataset)

# 평가 실행
evaluation_results = evaluate(
    formatted_dataset,
    metrics,
    model_id,
    embed_id,
    region
)

## 4. 결과 분석

In [ ]:
# 평가 결과 출력
print("\n평균 점수:")
for metric, score in evaluation_results['average_scores'].items():
    print(f"{metric}: {score:.4f}")

# 결과 저장
with open('evaluation_results.json', 'w', encoding='utf-8') as f:
    json.dump(evaluation_results, f, ensure_ascii=False, indent=4)